# Topic Modeling with gensim
We'll try out [Latent Dirichlet Allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) in [gensim](http://radimrehurek.com/gensim/index.html) on the [20 Newsgroups dataset](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html) with some simple preprocessing.

#### Install gensim

In [1]:
# pip install --upgrade gensim

##### imports

In [2]:
# gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Let's retain only a subset of the 20 categories in the original 20 Newsgroups Dataset.

In [3]:
# Set categories
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball']#, 
              #'rec.motorcycles', 'sci.space', 'talk.politics.mideast']
# Download the training subset of the 20 NG dataset, with headers, footers, quotes removed
# Only keep docs from the 6 categories above
ng_train = datasets.fetch_20newsgroups(subset='train', categories=categories, 
                                      remove=('headers', 'footers', 'quotes'))

In [4]:
# Take a look at the first doc
ng_train.data[0]

'\n\n\nI happen to be a big fan of Jayson Stark.  He is a baseball writer for the \nPhiladelphia Inquirer.  Every tuesday he writes a "Week in Review" column.  \nHe writes about unusual situations that occured during the week.  Unusual\nstats.  He has a section called "Kinerisms of the Week" which are stupid\nlines by Mets brodcaster Ralph Kiner.  Every year he has the LGTGAH contest.\nThat stands for "Last guy to get a hit."  He also writes for Baseball \nAmerica.  That column is sort of a highlights of "Week in Review."  If you \ncan, check his column out sometime.  He might make you laugh.\n\nRob Koffler\n'

## Document Preprocessing
We'll need to generate a term-document matrix of word (token) counts for use in LDA.

We'll use `sklearn`'s `CountVectorizer` to generate our term-document matrix of counts. We'll make use of a few parameters to accomplish the following preprocessing of the text documents all within the `CountVectorizer`:
* `analyzer=word`: Tokenize by word
* `ngram_range=(1,2)`: Keep all 1 and 2-word grams
* `stop_words=english`: Remove all English stop words
* `token_pattern=\\b[a-z][a-z]+\\b`: Match all tokens with 2 or more (strictly) alphabet characters

In [8]:
# Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")
count_vectorizer.fit(ng_train.data)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [11]:
# Create the term-document matrix
# Transpose it so the terms are the rows
counts = count_vectorizer.transform(ng_train.data).transpose()

In [17]:
counts.shape

(118113, 1661)

##### Convert to gensim
We need to convert our sparse `scipy` matrix to a `gensim`-friendly object called a Corpus:

In [12]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(counts)

##### Map matrix rows to words (tokens)
We need to save a mapping (dict) of row id to word (token) for later use by gensim:

In [20]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [21]:
len(id2word)

118113

## LDA
At this point we can simply plow ahead in creating an LDA model.  It requires our corpus of word counts, mapping of row ids to words, and the number of topics (3).

In [22]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=10)

2016-11-01 14:00:38,555 : INFO : using symmetric alpha at 0.3333333333333333
2016-11-01 14:00:38,562 : INFO : using symmetric eta at 0.3333333333333333
2016-11-01 14:00:38,564 : INFO : using serial LDA version on this node
2016-11-01 14:00:40,740 : INFO : running online LDA training, 3 topics, 10 passes over the supplied corpus of 1661 documents, updating model once every 1661 documents, evaluating perplexity every 1661 documents, iterating 50x with a convergence threshold of 0.001000
2016-11-01 14:00:55,523 : INFO : -17.439 per-word bound, 177635.9 perplexity estimate based on a held-out corpus of 1661 documents with 241620 words
2016-11-01 14:00:55,523 : INFO : PROGRESS: pass 0, at document #1661/1661
2016-11-01 14:00:59,844 : INFO : topic #0 (0.333): 0.001*don + 0.001*edu + 0.001*image + 0.001*people + 0.001*know + 0.001*good + 0.001*think + 0.001*use + 0.001*like + 0.001*year
2016-11-01 14:00:59,852 : INFO : topic #1 (0.333): 0.002*don + 0.002*like + 0.002*think + 0.002*just + 0.00

Let's take a look at what happened.  Here are the 5 most important words for each of the 3 topics we found:

In [23]:
lda.print_topics()

2016-11-01 14:04:44,583 : INFO : topic #0 (0.333): 0.003*image + 0.002*jpeg + 0.002*edu + 0.001*graphics + 0.001*file + 0.001*images + 0.001*don + 0.001*gif + 0.001*use + 0.001*pub
2016-11-01 14:04:44,587 : INFO : topic #1 (0.333): 0.002*don + 0.002*think + 0.002*just + 0.002*like + 0.002*god + 0.002*does + 0.002*good + 0.001*people + 0.001*know + 0.001*year
2016-11-01 14:04:44,591 : INFO : topic #2 (0.333): 0.002*don + 0.001*know + 0.001*just + 0.001*people + 0.001*like + 0.001*time + 0.001*god + 0.001*think + 0.001*does + 0.001*good


[(0,
  '0.003*image + 0.002*jpeg + 0.002*edu + 0.001*graphics + 0.001*file + 0.001*images + 0.001*don + 0.001*gif + 0.001*use + 0.001*pub'),
 (1,
  '0.002*don + 0.002*think + 0.002*just + 0.002*like + 0.002*god + 0.002*does + 0.002*good + 0.001*people + 0.001*know + 0.001*year'),
 (2,
  '0.002*don + 0.001*know + 0.001*just + 0.001*people + 0.001*like + 0.001*time + 0.001*god + 0.001*think + 0.001*does + 0.001*good')]

#### Topic Space
If we want to map our documents to the topic space we need to actually use the LdaModel transformer that we created above, like so:

In [27]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
lda_corpus

In [31]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

Now we can take a look at the document vectors in the topic space, which are measures of the component of each document along each topic.  Thus, at most a document vector can have num_topics=3 nonzero components in the topic space, and most have far fewer.

In [40]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:15]

[[(0, 0.99279555865152558)],
 [(1, 0.98246558383481875)],
 [(1, 0.98596682308976824)],
 [(0, 0.021380432710602391),
  (1, 0.95630391888657162),
  (2, 0.022315648402825867)],
 [(0, 0.98638820197097887)],
 [(1, 0.75274047903999508), (2, 0.24358700415759779)],
 [(1, 0.99390726605868784)],
 [(0, 0.011600764048619139),
  (1, 0.01131638137157851),
  (2, 0.97708285457980237)],
 [(0, 0.017617263280226312),
  (1, 0.96457892439353543),
  (2, 0.017803812326238171)],
 [(1, 0.9895168237363563)],
 [(1, 0.98537359074056685)],
 [(1, 0.98456791826605339)],
 [(0, 0.025951601254945982),
  (1, 0.029611964983043144),
  (2, 0.94443643376201092)],
 [(1, 0.99274143535311188)],
 [(1, 0.99282370712978607)]]

In [41]:
ng_train.data[5]

': In article <11838@vice.ICO.TEK.COM>, bobbe@vice.ICO.TEK.COM (Robert\n: >   Someone spank me if I\'m wrong, but didn\'t Lord, Liar, or Lunatic\n: >   originate with C.S. Lewis?  Who\'s this Campollo fellow anyway?\n\n: I do think so, and isn\'t there a clear connection with the "I do\n: believe, because it is absurd" notion by one of the original\n: Christians (Origen?).\n\nThere is a similar statement attributed to Anselm, "I believe so that\nI may understand". In both cases reason is somewhat less exalted than\nanyone posting here could accept, which means that neither statement\ncan be properly analysed in this venue.'

In [ ]:
lda.log_perplexity

## On your own...
- Go get some of the NIPS papers from [here](https://archive.ics.uci.edu/ml/datasets/Bag+of+Words).  
- Try performing LDA on this data with gensim
- Play with some of the preprocessing options and parameters for LDA, observe what happens
- See if you can use the resulting topic space to extract topic vectors and cluster some documents
- How do your results look?